# Fine-Tuning Llama 3.2 3B Instruct

In [ ]:
# %%capture
# %pip install -U transformers 
# %pip install -U datasets 
# %pip install -U accelerate 
# %pip install -U peft 
# %pip install -U trl 
# %pip install -U bitsandbytes 
# %pip install -U wandb

In [94]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [95]:
# login to huggingface cli 


In [ ]:
run = wandb.init(
    project='Fine-tune Llama 3.2 on code database', 
    job_type="training", 
    anonymous="allow"
)

In [97]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"
new_model = "llama-3.2-3b-code"
dataset_name = "code.csv"

In [ ]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    # %pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [99]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [100]:
#Importing the dataset
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load your dataset
data = pd.read_csv("code.csv", delimiter=";")  # Adjust delimiter if necessary

# Step 2: Split the dataset into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Save the train and test sets to CSV files
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

# Step 4: Load the datasets in Hugging Face format
train_dataset = load_dataset("csv", data_files={"train": "train.csv"})["train"]
test_dataset = load_dataset("csv", data_files={"test": "test.csv"})["test"]

instruction = """You are a programming expert specialized in reasoning and executing python algorithms. The user will provide you with a python code snippet and ask you to execute it with the given input. You should execute the code and provide the output to the user. If the code contains an error, you should identify the error and provide the user with a hint to fix it. If the code is correct, you should provide only the code output to the user."""

def format_chat_template(row):

    row_json = [{"role": "system", "content": instruction },
               {"role": "user", "content": row["python_code"]+ "\n "+ row["execution_example"]},
               {"role": "assistant", "content": row["execution_result"]}]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Load CSV files for train and test datasets separately
train_dataset = load_dataset('csv', data_files='train.csv', header=None)
test_dataset = load_dataset('csv', data_files='test.csv', header=None)

# Manually assign column names for both train and test datasets
train_dataset = train_dataset['train'].rename_column('0', 'python_code')
train_dataset = train_dataset.rename_column('3', 'human_reasoning')
train_dataset = train_dataset.rename_column('1', 'execution_example')
train_dataset = train_dataset.rename_column('2', 'execution_result')


test_dataset = test_dataset['train'].rename_column('0', 'python_code')
test_dataset = test_dataset.rename_column('3', 'human_reasoning')
test_dataset = test_dataset.rename_column('1', 'execution_example')
test_dataset = test_dataset.rename_column('2', 'execution_result')

train_dataset = train_dataset.map(format_chat_template, num_proc= 4)
test_dataset = test_dataset.map(format_chat_template, num_proc= 4)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/4040 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1011 [00:00<?, ? examples/s]

In [101]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [102]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

tokenizer.chat_template = None

model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [103]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=2,
    eval_strategy="steps",
    eval_steps=0.1,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=1e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [104]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/4040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1011 [00:00<?, ? examples/s]

In [105]:
trainer.train()

Step,Training Loss,Validation Loss
404,0.086500,0.071037
808,0.074000,0.066999
1212,0.084000,0.066622
1616,0.075700,0.066336
2020,0.050700,0.065122
2424,0.103700,0.065048
2828,0.073900,0.065012
3232,0.083200,0.064793
3636,0.080600,0.064298
4040,0.055800,0.064164


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home2/fbenslama/miniconda3/envs/deep_learning_torch/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class i

TrainOutput(global_step=4040, training_loss=0.0772970498862243, metrics={'train_runtime': 4080.4902, 'train_samples_per_second': 1.98, 'train_steps_per_second': 0.99, 'total_flos': 2.157149977430016e+16, 'train_loss': 0.0772970498862243, 'epoch': 2.0})

In [106]:
wandb.finish()

eval/loss,█▄▄▃▂▂▂▂▁▁
eval/runtime,█▄▃▃▆█▂▁▅▅
eval/samples_per_second,▁▅▆▆▃▁▇█▄▄
eval/steps_per_second,▁▅▆▆▃▁▇█▄▄
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇█
train/grad_norm,▃█▁▂▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█▇▇▇▇▇▇▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁
train/loss,▇█▃▂▆▆▄▆▄▂▁▆▆▃▄▄▃▂▄▇▄▃▇▂▂▃▇▂▂▂▃▂▇▃▂▄▂▄▄▄
eval/loss,0.06416
eval/runtime,130.9402


In [107]:
# # saving the fine-tuned model
# model.save_pretrained("farahbs/llama3.2_3b_instruct_simple_code_db")
# tokenizer.save_pretrained("farahbs/llama3.2_3b_instruct_simple_code_db")

# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load the fine-tuned model
# model = AutoModelForCausalLM.from_pretrained("farahbs/llama3.2_3b_instruct_simple_code_db")

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("farahbs/llama3.2_3b_instruct_simple_code_db")

In [108]:
# # pushing model and tokenizer to huggingface

# model.push_to_hub("llama3.2_3b_instruct_simple_code_db")
# tokenizer.push_to_hub("llama3.2_3b_instruct_simple_code_db")